# Case 1 -  Carregando arquivos

In [1]:
import pandas as pd
from pathlib import Path


In [2]:
path_data = 'Case 1 - Dados Fin/Dados Financeiros - Dados Recebidos.xlsx'
path_structure ='Case 1 - Dados Fin/Dados Financeiros - Estrutura HData.xlsx'

In [3]:
structure_file = pd.ExcelFile(path_structure)
structure_file.sheet_names

['Fluxo Caixa - Planilha Padrao',
 'Fluxo Caixa - Arq Completa',
 'dim_centro_custos',
 'dim_conta_financeira']

In [4]:
data_file = pd.ExcelFile(path_data)
data_file.sheet_names

['LANCAMENTO',
 'SALDOS INICIAIS',
 'DEPARA_CENTRO_CUSTOS',
 'DEPARA_CONTAS_FINANC']

In [5]:
base_arq = structure_file.parse('Fluxo Caixa - Planilha Padrao')

## Fase 1 - identificar a sequencia da tabela a ser montada a partir do arquivo base

In [6]:
base_arq['De Para'] = base_arq['De Para'].fillna('N/A')
base_arq['col de para entrada'] = base_arq['col de para entrada'].fillna('N/A')
base_arq['col de para saida'] = base_arq['col de para saida'].fillna('N/A')


In [7]:
#Pegando os dados da planilha de estrutura e separando por colunas, tabbelas e as informçãoes para fazer de para.
cols = base_arq[['Coluna Planilha','Aba Planilha','De Para','col de para entrada', 'col de para saida']].dropna()
cols_local = cols['Aba Planilha'].unique()

In [8]:
#Fazendo um dicionario com as infos da celula acima porem com a tabela como key.
cols_dict={}
for i in cols_local:
    cols_dict[i]=cols[cols['Aba Planilha']==i][['Coluna Planilha','De Para','col de para entrada', 'col de para saida']]

## Fase 2 - Limpando todas as bases de dados que temos exluindo arquivos duplicados e nulos

In [10]:
def clean_dup_null(data: pd.DataFrame) -> pd.DataFrame:
    data_rf = data.dropna(how='all').copy()
    data_rf = data_rf.drop_duplicates(keep='last').copy()
    return data_rf

In [11]:
data = {}
for j in data_file.sheet_names:
    data[j] = clean_dup_null(data_file.parse(j))

##  Fase 3 - Fazer os de para.

In [12]:
# COlunas divergentes que encontramos. Inserir caso alguma outra necessite de alteração.
# Key=Tabela de dados e Value = Tabela de estturuta
map_cols = {'LANCAMENTO':{'CONTA':'ANALÍTICO'}}

In [13]:
# Iteração para mudar o valor no dicionario de colunas o mapeamento acima.

for table,df in cols_dict.items():
    try:
        if len(map_cols[table]) > 0:
            for old,new in map_cols[table].items():
                data[table]=data[table].rename(columns={old:new})
    except:
        continue


In [14]:
# Filtrando os dados somente necessários e que precisam ser feito de para.
cols_d ={}
col_total={}
for table,df in cols_dict.items():
    df_filter = df[df['De Para']!= 'N/A'].copy()
    if not df_filter.empty:
        cols_d[table] = (df_filter.values.tolist())
    
    col_total[table]=data[table].columns.tolist()


In [34]:
# Aqui fazemos o de para. Iteramos nossas colunas e com isso filtramos elas nas bases de dados do nosso dict de dados.
# Apos isso é iterado novamente dentro das informações que necessitam de de para para fazer este de para
# Para automatização os valores de colunas para merge sao fornecidos na tabela de estrutura.
df_refined = pd.DataFrame()
for table,columns in col_total.items():
    df_refined= pd.concat([df_refined,data[table][columns]],axis=1)
    
    try:
        for k in cols_d[table]:
            col_left = k[0]
            table_right = k[1]
            col_left_base = k[2]
            col_target = k[3]
            data[table_right] = data[table_right].rename(columns={col_left_base:col_left})
            df_refined = df_refined.merge(data[table_right],on = col_left,how='left')
            df_refined[k[0]] = df_refined[k[3]]
    except:
        continue



In [29]:
#Fazendo lista com todos as colunas necessarias.
list_col_total=[]
for i,value in col_total.items():
    list_col_total += value

In [32]:
# DataFrame final com o dado trusted e automatizado pela orfganização do mapeamento feito.
df_trusted = df_refined[list_col_total]

In [35]:
# df_trusted a partir daqui pode se salvar ou enviar para onde for necessario a automatizer o processo